In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [40]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.reset_option('^display.', silent=True)

X_train = pd.read_csv("/content/drive/My Drive/HE/Dataset/Train.csv")
X_test = pd.read_csv("/content/drive/My Drive/HE/Dataset/Test.csv")

y_train=X_train.MULTIPLE_OFFENSE
X_train.drop(['MULTIPLE_OFFENSE'],axis=1,inplace=True)
X_train.head()


,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,CR_102659,04-JUL-04,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36
1,CR_189752,18-JUL-17,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34
2,CR_184637,15-MAR-17,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34
3,CR_139071,13-FEB-09,0,33,32,2,1,7,1,1,6,1,249,1.0,72,29,34
4,CR_109335,13-APR-05,0,33,32,2,1,8,3,0,5,1,174,0.0,112,29,43


In [41]:
print("Total training samples:", len(X_train), "\n")

Total training samples: 23856 



In [42]:
X_train.describe()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
count,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23674.000000,23856.000000,23856.000000,23856.000000
mean,0.483778,24.791206,24.637450,4.276744,2.455609,6.154175,4.876509,0.972460,4.924128,1.244802,206.954519,0.974064,85.237383,72.674296,33.464747
std,1.439738,15.240231,15.135093,2.944672,1.963095,4.471756,3.881931,1.453144,1.362625,1.119301,93.033348,1.167725,27.597226,43.297320,8.386834
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,7.000000,8.000000,2.000000,1.000000,3.000000,2.000000,0.000000,5.000000,1.000000,174.000000,1.000000,72.000000,29.000000,34.000000
50%,0.000000,24.000000,24.000000,4.000000,3.000000,5.000000,4.000000,1.000000,5.000000,1.000000,249.000000,1.000000,98.000000,62.000000,34.000000
75%,0.000000,36.000000,35.000000,6.000000,5.000000,8.000000,7.000000,1.000000,6.000000,1.000000,249.000000,1.000000,103.000000,107.000000,34.000000
max,7.000000,52.000000,52.000000,10.000000,5.000000,19.000000,18.000000,99.000000,6.000000,90.000000,332.000000,90.000000,116.000000,142.000000,50.000000


In [43]:
X_train.iloc[25]

INCIDENT_ID     CR_73049
DATE           28-MAY-01
X_1                    0
X_2                    4
X_3                    4
X_4                    6
X_5                    5
X_6                    3
X_7                    5
X_8                    0
X_9                    5
X_10                   1
X_11                 174
X_12                   1
X_13                  72
X_14                  80
X_15                  34
Name: 25, dtype: object

In [44]:
X_train["DATE"] = pd.to_datetime(X_train["DATE"]).dt.strftime("%Y%m%d").astype(int)
X_test["DATE"] = pd.to_datetime(X_test["DATE"]).dt.strftime("%Y%m%d").astype(int)
X_train['INCIDENT_ID'] = pd.to_numeric(X_train.INCIDENT_ID.str.replace('CR_','')).astype(int)
y_test_ids = X_test['INCIDENT_ID']
X_test['INCIDENT_ID'] = pd.to_numeric(X_test.INCIDENT_ID.str.replace('CR_','')).astype(int)
X_train.head()


,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,102659,20040704,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36
1,189752,20170718,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34
2,184637,20170315,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34
3,139071,20090213,0,33,32,2,1,7,1,1,6,1,249,1.0,72,29,34
4,109335,20050413,0,33,32,2,1,8,3,0,5,1,174,0.0,112,29,43


In [45]:
y_test_ids.head()

0    CR_195453
1    CR_103520
2    CR_196089
3    CR_112195
4    CR_149832
Name: INCIDENT_ID, dtype: object

In [46]:
X_test.head()

,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,195453,20180201,0,30,35,7,3,6,4,0,5,1,174,NaN,72,119,23
1,103520,20040305,0,44,44,1,3,7,1,4,6,1,316,0.0,12,29,34
2,196089,20180127,0,34,33,3,5,2,7,3,0,1,316,1.0,72,0,34
3,112195,20060818,7,3,2,3,5,9,8,0,5,1,174,1.0,112,87,34
4,149832,20111031,0,7,8,7,3,2,7,1,5,1,174,0.0,112,93,43


In [47]:
X_train.dtypes.value_counts()
categorical_columns = X_train.select_dtypes('object').columns
print(len(X_train.columns)-len(X_train.select_dtypes('object').columns),'numerical columns:')
print([i for i in list(X_train.columns) if i not in list(X_train.select_dtypes('object').columns)], '\n')
print(len(X_train.select_dtypes('object').columns),'categorical columns:')
print(list(X_train.select_dtypes('object').columns))

17 numerical columns:
['INCIDENT_ID', 'DATE', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15'] 

0 categorical columns:
[]


In [48]:
from sklearn.impute import SimpleImputer
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

X_train_missing_num = [col for col in X_train.columns if X_train[col].isnull().any() and is_numeric_dtype(X_train[col])]
X_test_missing_num = [col for col in X_test.columns if X_test[col].isnull().any() and is_numeric_dtype(X_test[col])]

print('X_train num missing cols:', X_train_missing_num)
print('X_test num missing cols:', X_test_missing_num)

X_train num missing cols: ['X_12']
X_test num missing cols: ['X_12']


In [50]:
object_cols = ['INCIDENT_ID', 'DATE']
X_train_obj = X_train[object_cols]
X_test_obj = X_test[object_cols]
X_train.drop(object_cols, axis=1, inplace=True)
X_test.drop(object_cols, axis=1, inplace=True)
si = SimpleImputer()
X_train_imputed = pd.DataFrame(si.fit_transform(X_train))
X_test_imputed = pd.DataFrame(si.transform(X_test))

X_train_imputed.columns = X_train.columns
X_test_imputed.columns = X_test.columns

X_train = pd.merge(X_train_imputed, X_train_obj, left_index=True, right_index=True)
X_test = pd.merge(X_test_imputed, X_test_obj, left_index=True, right_index=True)

print(X_train.isnull().values.any())
print(X_test.isnull().values.any())

False
False


In [51]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

N_JOBS=24
rs = np.random.RandomState(2)

# Train and submit a RBF SVM classfier using best parameters found
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4),
                          random_state=rs, n_estimators=200,learning_rate=0.01)

clf.fit(X_train_scaled, y_train)
test_preds = clf.predict(X_test_scaled)
print(test_preds)


output = pd.DataFrame({'INCIDENT_ID': y_test_ids,
                       'MULTIPLE_OFFENSE': test_preds})
output.to_csv('submission2.csv', index=False)

[1 1 1 ... 1 1 1]
